In [1]:
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import yfinance as yf

In [2]:
wiki = 'https://en.wikipedia.org/wiki/'

In [3]:
cac40_tickers = pd.read_html(wiki + 'CAC_40', flavor='html5lib')[4]['Ticker'].to_list()

In [4]:
def get_stock_price_history(tickers):
    data = []
    for ticker in tickers:
        data.append(yf.download(tickers=ticker,
                                period="1mo",
                                interval="5m").reset_index())
    return data

In [5]:
cac40 = get_stock_price_history(cac40_tickers)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
def new_engine(name):
    engine = create_engine(f'postgresql://pavelkurach@localhost:5432/{name}')
    if not database_exists(engine.url):
        create_database(engine.url)
    print(database_exists(engine.url))
    return engine

In [7]:
cac40_engine = new_engine('CAC40')

True


In [8]:
def to_sql(frames, tickers, engine):
    for frame, ticker in zip(frames, tickers):
        frame.to_sql(ticker, engine, if_exists='replace', index=False)
        with cac40_engine.connect() as con:
            con.execute(f'ALTER TABLE "{ticker}" ADD PRIMARY KEY ("{frame.columns[0]}");')

    print("Successfully created a database")

In [9]:
to_sql(cac40, cac40_tickers, cac40_engine)

Successfully created a database


In [10]:
Base = automap_base()

Base.prepare(autoload_with=cac40_engine)

In [11]:
cac40_classes = {ticker: cls for ticker, cls in zip(cac40_tickers, Base.classes)}

In [12]:
session = Session(cac40_engine)

In [13]:
query = session.query(cac40_classes['AI.PA'])

In [14]:
pd.read_sql(query.statement, query.session.bind)

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2022-11-22 09:00:00,346.299988,347.549988,346.000000,346.649994,346.649994,0
1,2022-11-22 09:05:00,346.600006,346.750000,345.149994,345.250000,345.250000,2748
2,2022-11-22 09:10:00,345.350006,345.899994,344.799988,345.200012,345.200012,2161
3,2022-11-22 09:15:00,345.200012,346.299988,345.100006,345.750000,345.750000,2629
4,2022-11-22 09:20:00,345.700012,345.700012,343.850006,344.200012,344.200012,6499
...,...,...,...,...,...,...,...
2240,2022-12-21 17:10:00,341.100006,341.250000,340.950012,341.000000,341.000000,3068
2241,2022-12-21 17:15:00,341.000000,341.100006,340.850006,340.950012,340.950012,2072
2242,2022-12-21 17:20:00,340.899994,341.000000,340.200012,340.450012,340.450012,2959
2243,2022-12-21 17:25:00,340.450012,340.549988,339.950012,340.200012,340.200012,2775
